Probeer deze eerste cell te runnen om te testen of je notebook werkt. Als deze de output Hello world geeft, werkt het.
(je kan een cell uitvoeren door deze te selecteren en SHIFT+ENTER te gebruiken, of door op de play knop te klikken).

In [ ]:
print('Hello world!')

**Jupyter notebooks**
Wat heb ik voor me? Dit is een Jupyter notebook. Dit is eigenlijk een omgeving waarin je python code stukje voor stukje kan draaien. Variabelen en functies worden bewaard tussen de cellen, dus je hoeft niet steeds je hele script opnieuw te draaien wanneer je een nieuw stukje code hebt gegenereerd. Dit notebook bestaat uit markdown cellen met tekst en uitleg, en code cellen met python code. Elke code cell heeft "In [ ]:" er voor staan. De code die je hier invoert, wordt als input gezien, direct onder de cell verschijnt de output.

_Jupyter heeft bepaalde styling ingebouwd. Als je aan het eind van je cell een bepaalde waarde wilt outputten, hoef je deze niet tussen print() statement te zetten, zolang je de naam van de variabele gewoon op de laatste regel van je cell plaatst. In sommige gevallen, zoals met pandas dataframes, zal je output er mooier uit zien zonder print statement, omdat jupyter automatisch styling toepast. (let wel op, als je je jupyter code naar ene python document kopieert zal de code in de war raken van willekeurige rijen met alleen de naam van een variabele er in)_

# De Titanic
In 1912 verging de Titanic als gevolg van een ijsberg. Nu, ruim 100 jaar later, kunnen we terugkijken op de slachtoffers en overlevenden. Er is openbare data beschikbaar over de passagiers en hun lot, en deze kunnen we gebruiken om te voorspellen wie de reis wel of niet zou overleven als ze er bij waren geweest.

In deze workshop gaan we aan de hand van de [Titanic dataset](https://public.opendatasoft.com/explore/dataset/titanic-passengers/export/) doornemen hoe machine learning werkt.

In [ ]:
# processing
import numpy as np
import pandas as pd

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn import tree

We gaan nu beginnen met het maken van ons model. Om te beginnen hebben we een aantal libraries nodig. Pandas en numpy voor het werken met een dataset en wiskundige berekeningen, matplotlib en seaborn voor visualisatie van data, en sklearn voor voorgebouwde algoritmes en databewerking.

Zoals je ziet geef ik de meeste libraries die ik importeer direct een alias. Deze zijn allemaal volgens standaard best practices, dus deze afkortingen zul je vaak tegenkomen op internet.

# Extract
We beginnen met het inladen van onze data. Er zijn meerdere versies van de Titanic dataset te vinden. Globaal bevatten ze allemaal ongeveer de zelfde kolommen en waardes. Wij gebruiken deze: [bron](https://public.opendatasoft.com/explore/dataset/titanic-passengers/export/). Ik heb de csv al toegevoegd aan deze repository, dus we gaan de dataset importeren uit onze eigen files.


In [ ]:
df = pd.read_csv('titanic-passengers.csv', delimiter=';')
df.head()

## Inventarisatie van de data
`df.head()` geeft ons de eerste 5 rijen data uit de dataset. Hierin zien we alle kolommen, en direct wat voorbeelden van wat voor soort data er in zit.
- PassengerId: uniek id voor elke passagier
- Survived
- Pclass: de klasse waarin de passagier heeft gereisd (1, 2 of 3)
- Name
- Sex
- Age
- SibSp: aantal siblings / spouses aan boord
- Parch: aantal parents / children aan boord
- Ticket: ticketnummer
- Fare: het bedrag wat de passagier heeft betaald
- Cabin: kamernummer waar de passagier verbleef, hieruit kunnen we halen welke Deck een passagier zat
- Embarked: de haven waar de passagier aan boord van het schip is gekomen

## Wat zijn onze verwachtingen? 
Nu we weten welke data we hebben, wat verwachten we dat de grootste impact zal hebben? We kennen allemaal het verhaal van de Titanic, hebben mogelijk de film gezien. Op basis van deze voorkennis, kunnen we onze eerste hypotheses opstellen. We verwachten bijvoorbeeld dat de volgende factoren de groote impact zullen hebben op de overlevingskans van een passagier: leeftijd, geslacht, en klasse.
Deze hypotheses zullen we zometeen checken.


## Een eerste verkenning van de data
Voordat we de data gaan analyseren, willen we checken of datatypes goed zijn geïmporteerd:

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df[['Survived', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']].describe()

### Verdeling van waardes over de data
`df.describe()` geeft wat snelle geaggregeerde data per numerieke kolom. We zien het aantal waardes (waaruit opvalt dat `Age` een paar NULL (NaN) waardes heeft), het gemiddelde, standaarddeviatie, minimum en maximum, en het 25e, 50e en 75e percentiel. Zo weten we hoe de verdeling is van de waardes. We kunnen hier een aantal conclusies uit trekken over de datakwaliteit:

- Passenger Id: dit zegt ons weinig. Een id geeft geen specifieke waarde over een passagier (passagier 20 is bijvoorbeeld niet tweemaal zo veel waard als passagier 10)
- Passenger Class: het 50e percentiel heeft al de zelfde waarde als de maximum waarde. Hieraan zien we dat meer dan de helft van de pasagiers 3e klas reisde.
- Age: de count is hier afwijkend van de rest, er zitten dus een aantal missende waardes in deze kolom.
- SibSp: we zien dat ruim de helft van de passagiers geen siblings of spouses aan boord had. De afstand tussen het 95e percentiel en de maximale waarde is erg groot, dus er zijn enkele outliers, maar de meeste mensen hebben geen of 1 sibling/spouse bij zich.
- Parch: we zien dat weinig mensen hun kinderen of ouders aan boord hadden, met enkele uitschieters.
- Fare: we zien dat er passagiers zijn die een fare hebben betaald van 0, mogelijk waren dit werknemers op het schip? De gemmiddelde fare was 32, met een standaarddeviatie van 49. Dit suggereert dat er veel verschillen zaten tussen de fares van passagiers. We zien ook dat het 75e percentiel al bijna op de waarde van het gemmiddelde zit. Dit geeft aan dat het gemmiddelde ver omhoog wordt getrokken dankzij enkele hoge outliers.
- Survived geeft aan dat het meest voorkomende geval "No" is, ofwel een meerderheid van 549 passagiers heeft de reis niet overleefd.
- Sex laat ons zien dat er iets meer mannelijke passagiers aanwezig waren dan vrouwelijke.
- De Cabin kolom mist veel data. We hebben maar 204 rijen beschikbaar, dus dat is lastig te corrigeren. Deze data zullen we niet kunnen gebruiken. 
- Embarked mist twee waardes. Deze zullen we later op kunnen vullen.

Nu we deze waardes hebben gezien, kunnen we nog verder duiken in de verdeling van de waardes. Zoals je ziet hebben we hierboven in de `.describe()` onderscheid gemaakt tussen numerieke en tekstuele kolommen. Om te kunnen kijken naar al deze kolommen t.o.v. de overleving van de passagier, maken we een nieuwe kolom waarin we Survived omzetten naar een getal.

In [ ]:
df.loc[df['Survived'] == "Yes", 'alive'] = 1
df.loc[df['Survived'] == "No", 'alive'] = 0

In [ ]:
pd.plotting.scatter_matrix(df, figsize=(15,15), alpha=0.2)

Uit deze scatterplots en histogrammen komen een aantal interessante waarnemingen naar boven. We zien terug dat 
- de meerderheid de ramp niet overleefd heeft (rechtsonder)
- de Fare inderdaad vooral concentreert rond lage prijzen, maar enkele uitschieters heeft boven de 400
- we zien dat tussen Age en Pclass, de derde klas passagiers over het algemeen een lagere leeftijd hebben dan de eerste klas passagiers

Dat laatste gaan we direct controleren:

In [ ]:
df[['Pclass', 'Age']].groupby(['Pclass']).agg(['mean', 'min', 'max', 'std', 'var'])

De gemmiddelde leeftijd (de mean) van derde klas passagiers ligt meer dan 10 jaar lager dan eerste klas! Daarnaast zien we dat de oudste passagier (80 jaar) eerste klas reisde. Aan de standaarddeviatie zien we dat er onder de eerste klas meer variatie in leeftijd bestond dan onder de derde klas.

### Hypotheses controleren
We hadden ingeschat dat leeftijd, geslacht en klasse grote voorspellers zouden zijn voor overlevingskans. Klopt dat uit de data? We maken een paar grafieken om dat te testen.

In [ ]:
print(df[['Pclass', 'alive']].groupby(['Pclass']).mean())
plt.figure()
df['Pclass'].loc[df['alive'] == 0].hist(alpha=0.4, label='died')
df['Pclass'].loc[df['alive'] == 1].hist(alpha=0.4, label='survived')
plt.title('Pclass distribution over survival')
plt.legend()
plt.show()

In [ ]:
df[['Sex', 'alive']].groupby(['Sex']).mean()

In [ ]:
df.loc[df['Age'] >= 18, 'adult'] = 'Yes'
df.loc[df['Age'] < 18, 'adult'] = 'No'
print(df[['adult', 'alive']].groupby(['adult']).agg(['count', 'mean']))
plt.figure()
df['Age'].loc[df['alive'] == 0].hist(alpha=0.4, label='died')
df['Age'].loc[df['alive'] == 1].hist(alpha=0.4, label='survived')
plt.title('Age distribution over survival')
plt.legend()
plt.show()

Op basis van de data lijken onze hypotheses te kloppen. 

**Klasse**
Van de eerste klas passagiers heeft een grote meerderheid de ramp overleefd, in de tweede en derde klas verschuift deze meerderheid naar de niet-overlevers. Voornamelijk de derde klasse passagiers liepen risico: maar 24% van hen heeft overleefd.

**Geslacht**
Vrouwen kregen voorrang op de reddingsboten, en dat zien we. Van de vrouwen heeft bijna 75% het overleefd, onder de mannen was het percentage overlevers maar 18%.

**Leeftijd**
We schatten in dat kinderen, net als vrouwen, voorrang kregen in de reddingsboten. Dit kunnen we het makkelijkst procentueel weergeven door een splitsing te maken op volwassenheid, dus we hebben een kolom `adult` gemaakt, op leeftijd hoger of lager dan 18 jaar oud. Hieruit blijkt dat bijna 54% van de kinderen de ramp heeft ovrleefd, en maar 38% onder volwassenen.

Onze hypotheses lijken dus te kloppen uit de data. We zullen straks zien of het model deze zelfde kenmerken aan zal wijzen als belangrijkste invloeden.

# Transform

Nu komen we bij de volgende fase. We hebben gezien dat we in sommige kolommen wat data missen, soms zoveel dat de data onbruikbaar wordt. Daarnaast hebben we kolommen gezien met data die voor ons niet relevant is, ook deze zullen we verwijderen.

In sommige gevallen kunnen we missende data wel opvullen, en daar zullen we dat doen.

## Data cleanen

### Niet relevante data

We hebben gezien dat de Passenger_id kolom voor ons niet relevant is. Deze zullen we verwijderen uit het dataframe. Dit zelfde geldt voor het ticketnummer.

### Missende data

Missende data kunnen we op verschillende manieren behandelen:
- De waarde alsnog achterhalen: is het mogelijk deze waardes alsnog te vinden, bij de bron van je dataset of op internet?
- De waarde achterhalen uit andere data, mogelijk zit er in een andere kolom informatie waarmee je deze waarde alsnog kan vinden.
- De waarde opvullen. Dit kan mogelijk uit de andere waardes in de zelfde kolom. Zo zou je bijvoorbeeld het gemmiddelde of de modus (meest voorkomende waarde) of de mediaan (middelste waarde) kunnen gebruiken, en daarmee de missende velden vullen.
- De hele rij verwijderen: als deze kolom erg relevant is voor je data, en je kan de waarde niet met genoeg vertrouwen opvullen, is het misschien nodig om de hele rij te verwijderen. Als er in de kolom in te veel rijen waardes missen, is het lastig die op te vullen en kan de kolom mogelijk niet worden meegenomen in het model.

De kolom Embarked mist maar een paar waardes. We kunnen niet achterhalen in welke haven een passagier is opgestapt, maar in dit geval kunnen we veilig genoeg de waarde opvullen met de modus van de kolom.

De kolom Cabin mist een grote meerderheid van de waardes. We kunnen dit niet achterhalen uit een andere kolom, we weten niet genoeg om de waardes op te vullen met de modus of mediaan, dus we zullen deze kolom verwijderen.

De kolom Age mist ook veel waardes. Deze kolom is voor ons waarschijnlijk erg interessant, dus deze willen we niet verwijderen. We zullen deze in dit geval opvullen met het gemmiddelde van de kolom. (Kun je nog andere tactieken bedenken? Probeer die eens uit).

In [ ]:
df.drop(['PassengerId'], axis=1, inplace=True)

In [ ]:
df.drop(['Ticket'], axis=1, inplace=True)

In [ ]:
df.drop(['Cabin'], axis=1, inplace=True)

In [ ]:
df['Embarked'].value_counts()

In [ ]:
df['Embarked'] = df['Embarked'].fillna('S')

In [ ]:
df['Age'] = df['Age'].fillna(df['Age'].mean())

## Data voorbereiden

Nu de data schoon is, moeten we nog wat bewerkingen doen om deze in het model te kunnen laden. Hierin zijn in dit geval twee stappen relevant:

### One-Hot-Encoding
We hebben een paar kolommen in tekstvorm. Een model kan niet zomaar rekenen met tekst, dus deze willen we omzetten naar numerieke waardes. Dit kunnen we echter niet zomaar doen met id's. Een voorbeeld: stel we hebben data gelinkt aan een tijdsverloop, met daarin een kolom `dag_van_de_week`. We kunnen dit niet zomaar omzetten naar `maandag = 1, dinsdag = 2, etc..`. Dinsdag is namelijk niet tweemaal zoveel waard als maandag, en je kan niet dinsdag maal 3 doen om op zaterdag uit te komen.

Daarom zouden we de kolom met de dag van de week omzetten naar een nieuwe kolom voor elke categorie. Zo krijgen we nieuwe kolommen: [dvdw_maandag, dvdw_dinsdag, dvdw_woensdag, etc..]. In elk van deze kolommen staat een 0 of 1, afhankelijk van welke waarde voor die rij relevant is. Wanneer een bepaalde rij dus bij maandag hoort, is de eerste kolom een 1, en alle andere zijn 0. De originele kolom met de labels gebruiken we niet meer.

Nadat we dit hebben gedaan, moeten we nog één stap nemen: het verwijderen van exact 1 van de nieuwe kolommen. Zo zouden we in het voorbeeld bijvoorbeeld dvdw_zondag weghalen. Dit voorkomt afhankelijkheid onder onze onafhankelijke variabelen. Als we alle kolommen laten bestaan, zal uit de waardes in 6 kolommen altijd met zekerheid de waarde van de 7e kolom blijken. Dit verstoort ons model.

Dit hele proces noemen we **One-Hot-Encoding**.

In het voorbeeld van de titanic hebben we een aantal tekst kolommen: 'Name', 'Sex', 'Embarked'. Binnen de kolom Name hebben we erg veel verschillende waardes, en de naam van een persoon zal weinig zeggen over zijn of haar overlevingskans, dus deze kolom zullen we niet gebruiken. (Kun jij een manier bedenken om deze kolom wel relevant te maken? Probeer het uit en test de invloed op het model!)

De kolom Sex kunnen we eenvoudig omzetten naar een numerieke kolom `female` met daarin 1 of 0.

De kolom Embarked bevat drie verschillende waardes. Deze zullen we omzetten naar nieuwe kolommen, met een pandas methode die hiervoor gemaakt is.

Daarnaast hebben we numerieke kolommen die geen numerieke waarde representeren: bijvoorbeeld Pclass.
Klasse 1 is niet de helft van klasse 2 waard (als deze verhouding al bestaat, dan zou die andersom zijn. Wil je testen of deze bestaat? Sla dan een stap over hier en test het model).
We zullen nu Pclass behandelen als tekstkolom en deze ook omzetten naar nieuwe kolommen per categorie.

In [ ]:
df.drop(['Name'], axis=1, inplace=True)

In [ ]:
gendermap = {'male': 0, 'female': 1}
df['female'] = df['Sex'].map(gendermap)
df.drop(['Sex'], axis=1, inplace=True)

In [ ]:
df = pd.get_dummies(df, prefix=['Embarked'], columns=['Embarked'])
df.drop(['Embarked_C'], axis=1, inplace=True)

In [ ]:
df = pd.get_dummies(df, prefix=['Pclass'], columns=['Pclass'])
df.drop(['Pclass_1'], axis=1, inplace=True)

In [ ]:
# save copy for later
df2 = df.copy()

In [ ]:
# We hebben tijdens het verkennen en cleanen van de data een paar nieuwe kolommen toegevoegd, die we nu niet zullen gebruiken, dus die halen we weer weg.
df.drop(['alive'], axis=1, inplace=True)
df.drop(['adult'], axis=1, inplace=True)

# Laten we kijken wat er uiteindelijk van onze dataset overblijft.
df.head()

# Model trainen

Nu is het zo ver! We gaan een model trainen. Om dit te doen, maken we eerst de splitsing tussen onze training en test data.

We gaan uit van een seed van 7. Dit is willekeurig, en kun je aanpassen. We gebruiken nu een vast getal, zodat we allemaal de zelfde splitsing krijgen, maar je kan deze vervangen door een ander getal, of door een random nummer, om andere willekeurige splitsingen te krijgen.

Daarnaast gaan we voor een test_size van 20%. Ook dit kun je eventueel aanpassen.

In [ ]:
# split data into training features and labels
X, y = df.loc[:, df.columns != 'Survived'], df['Survived']

# split data into train and test sets
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

## Decision Tree
In dit geval gebruiken we een decision tree classifier. Dit algoritme maakt een beslisboom model waaruit een zo goed mogelijke splitsing komt. Dit is een van de modellen die al is ingebouwd in `sklearn`.

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

## Voorspellen
Nu we een getraind model hebben, kunnen we een voorspelling maken op onze test_data, om te kijken hoe goed het model presteert. We maken eerst een voorspelling, en daarna plaatsen we deze in een confusion matrix. Hierin kunnen we het aantal True Positives, True Negatives, False Positives en False Negatives lezen.

In [ ]:
Y_pred = decision_tree.predict(X_test)

In [ ]:
decision_tree.score(X_test, y_test)

In [ ]:
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(decision_tree, X_test, y_test,
                                 display_labels=['Yes', 'No'],
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

## Model analyseren
Sommige machine learning modellen worden wel eens "black box" genoemd. Bijvoorbeeld de google en facebook algoritmes schijnen onbegrijpbaar te zijn voor mensen, puur het resultaat van heel veel training data in een model laden wat niet uit te lezen is.

Gelukkig is een decision tree dat wel. We kunnen het getrainde model uitwerken om te zien welke volgorde van parameters het model heeft gekozen om een voorspelling te maken.

In [ ]:
fn=list(X_train.columns)
cn=['Yes', 'No']
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (10,10), dpi=300)
tree.plot_tree(decision_tree,
               feature_names = fn, 
               class_names=cn,
               filled = True);
fig.savefig('decisiontree_visual.png')

# Model tweaken

We hebben nu een model wat een score haalt van 74% correcte voorspellingen. Dit is best oké, maar kan beter! We kunnen onze parameters aanpassen om een beter model te maken. Heb je ideeën hoe?

We zouden onze data verder kunnen parsen tot andere waardes. Denk aan: Age categoriseren (bijv '0-20', '20-40', etc) (of: adult: 1 of 0). Mogelijk hebben we nu ook data meegenomen die helemaal niet zo interessant is voor ons model. In het model hierboven valt bijvoorbeeld op dat 'Embarked' vaak pas als laatste criterium wordt gebruikt. Wat gebeurt er met het model wanneer we die kolom weglaten?

Daarnaast kunnen we kijken naar het reisgezelschap, had je meer kans op overleving als je alleen reist of met familie?
En je [titel](https://www.kaggle.com/manuelatadvice/feature-engineering-titles)?

Al deze aanpassingen kunnen we doen, het model opnieuw draaien, meer aanpassingen doen, enz. (let op, je kan deze code niet zomaar draaien nu, omdat we bijvoorbeeld de 'Name' kolom bovenin al hebben verwijderd. Wil je de data tweaken, draai dan dit notebook opnieuw vanaf het inladen van de csv file en pas de relevante stappen aan, of sla stappen over. Draai daarna het model opnieuw.

In [ ]:
# df['alone'] = np.where((df['SibSp'] == 0) & (df['Parch'] == 0), 1, 0)

In [ ]:
# df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

## Onze hypotheses

Laten we eens wat proberen. We hebben dit stiekem al voorbereid toen we een kopie maakten van de dataset eerder, deze gaan we nu gebruiken. We gaan even puur de kolommen gebruiken die we zelf relevant hadden verwacht: leeftijd, geslacht en klasse. Om het model super simpel te maken, splitsen we leeftijd uit in 2 categoriën: volwassen of kind. (Omdat dit een categoriale waarde is, One-Hot-Encoden we die ook meteen).
Daarna draaien we het hele model nog eens, en bekijken de impact van deze parameters:

In [ ]:
df2.head()

In [ ]:
df2.loc[df2['Age'] >= 18, 'adult'] = 1
df2.loc[df2['Age'] < 18, 'adult'] = 0
df2.drop(['Age', 'SibSp', 'Parch', 'Fare', 'alive', 'Embarked_Q', 'Embarked_S'],axis=1, inplace=True)
df2.head()

In [ ]:
# split data into training features and labels
X, y = df2.loc[:, df2.columns != 'Survived'], df2['Survived']

# split data into train and test sets
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

Y_pred = decision_tree.predict(X_test)

print('score:', decision_tree.score(X_test, y_test))

titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(decision_tree, X_test, y_test,
                                 display_labels=['Yes', 'No'],
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

from sklearn import tree
fn=list(X_train.columns)
cn=['Yes', 'No']
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (5,5), dpi=300)
tree.plot_tree(decision_tree,
               feature_names = fn, 
               class_names=cn,
               filled = True);
fig.savefig('decisiontree_visual.png')

Onze score is gestegen! We zijn van 74% naar 83% gegaan, met een veel simpeler model. Daarnaast blijkt uit de visualisatie van ons model welke waardes het meest interessant zijn. Lees het model maar eens door!

We zien dat de belangrijkste keuze geslacht is. Is je geslacht man (female <= 0.5), dan hangt je overleving daarna af van je leeftijd. Is je geslacht echter vrouw, dan hangt je geslacht daarna het meest af van in welke klasse je reist, en in het geval dat je 3e klas reis, je leeftijd. Reis je als vrouw 2e of 1e klas? Dan lijkt uit het model je kans op overleving eigenlijk al heel klein! Klopt dat?

# Zou jij overleven?

Nog één laatste test. Zou jij de Titanic ramp overleefd hebben? Gezien jouw leeftijd en geslacht, welke klasse zou je moeten reizen om kans te hebben op overleving?

In [ ]:
test_jezelf = pd.DataFrame([{'adult': 1, 'female': 1, 'Pclass_2': 0, 'Pclass_3': 0}])
# let op dat je Pclass_1 er bij moet denken, maar niet moet toevoegen. Één van de Pclass kolommen moet 1 zijn, de andere 2 hebben dan een 0.
test_jezelf

In [ ]:
decision_tree.predict(test_jezelf)